## Dependencies

In [1]:
import glob
import warnings
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling1D, SpatialDropout1D, Concatenate, Add
from googleqa_utilityscript import *
from transformers import BertConfig, BertTokenizer, TFBertModel


SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
BERT_PATH = '/kaggle/input/bert-base-uncased-huggingface-transformer/bert-base-uncased-tf_model.h5'
VOCAB_PATH = '/kaggle/input/bert-base-uncased-huggingface-transformer/bert-base-uncased-vocab.txt'
model_path_list = glob.glob('/kaggle/input/144-googleq-a-train-3fold-bert-base-unc-sum/' + '*last_epoch.h5')
model_path_list.sort()
print('Models to predict:')
print(*model_path_list, sep = "\n")

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Models to predict:
/kaggle/input/144-googleq-a-train-3fold-bert-base-unc-sum/model_fold_1_last_epoch.h5
/kaggle/input/144-googleq-a-train-3fold-bert-base-unc-sum/model_fold_2_last_epoch.h5
/kaggle/input/144-googleq-a-train-3fold-bert-base-unc-sum/model_fold_3_last_epoch.h5
Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

## Pre-process data

In [4]:
text_features = ['question_title', 'question_body', 'answer']

# Model parameters

In [5]:
N_CLASS = len(target_cols)
MAX_SEQUENCE_LENGTH = 512

## Test set

In [6]:
tokenizer = BertTokenizer.from_pretrained(VOCAB_PATH, do_lower_case=True)

# Test features
X_test = compute_input_arays(test, text_features, tokenizer, MAX_SEQUENCE_LENGTH)

# Model

In [7]:
bert_config = BertConfig()
bert_config.output_hidden_states=True

def model_fn():
    input_word_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids')
    input_masks = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks')
    segment_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids')

    bert_model = TFBertModel.from_pretrained(BERT_PATH, config=bert_config)
    sequence_output, pooled_output, hidden_states = bert_model([input_word_ids, input_masks, segment_ids])
    
    h12 = tf.reshape(hidden_states[-1][:,0],(-1,1,768))
    h11 = tf.reshape(hidden_states[-2][:,0],(-1,1,768))
    h10 = tf.reshape(hidden_states[-3][:,0],(-1,1,768))
    h09 = tf.reshape(hidden_states[-4][:,0],(-1,1,768))
    
    x = Add()([h12,h11, h10, h09])

    x = GlobalAveragePooling1D()(x)
    x = Dropout(0.2)(x)
    output = Dense(N_CLASS, activation="sigmoid", name="output")(x)

    model = Model(inputs=[input_word_ids, input_masks, segment_ids], outputs=output)
    
    return model

# Make predictions

In [8]:
Y_test = np.zeros((len(test), N_CLASS))

for model_path in model_path_list:
    model = model_fn()
    model.load_weights(model_path)
    Y_test += model.predict(X_test) / len(model_path_list)

In [9]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)
display(submission.head())
display(submission.describe())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.943783,0.661348,0.252302,0.484175,0.469319,0.457126,0.699449,0.640520,0.532189,...,0.920437,0.910231,0.604772,0.955549,0.945943,0.819107,0.027143,0.037745,0.901408,0.927992
1,46,0.887400,0.490653,0.006524,0.788967,0.741434,0.894204,0.548247,0.463783,0.076159,...,0.720309,0.941901,0.644292,0.961545,0.975508,0.869256,0.922838,0.119144,0.089639,0.888467
2,70,0.924992,0.610069,0.026440,0.729200,0.885021,0.918004,0.614863,0.479880,0.168140,...,0.889280,0.931446,0.649865,0.961394,0.967681,0.849012,0.086284,0.056121,0.865973,0.930202
3,132,0.876853,0.433461,0.008820,0.719056,0.741565,0.866051,0.588775,0.462050,0.102446,...,0.741157,0.939153,0.659517,0.961563,0.971216,0.885385,0.793115,0.154222,0.543677,0.897950
4,200,0.952160,0.473855,0.030485,0.858402,0.675063,0.796285,0.643911,0.579650,0.051424,...,0.728847,0.914388,0.616451,0.959820,0.967697,0.841928,0.264266,0.106076,0.575642,0.918513


,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,...,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,5029.186975,0.894322,0.585153,0.035572,0.747548,0.803840,0.818311,0.588436,0.487293,0.213442,...,0.817599,0.930364,0.657049,0.956165,0.967173,0.869540,0.503004,0.121077,0.497863,0.913667
std,2812.670060,0.047954,0.140289,0.074721,0.099136,0.113463,0.148020,0.052071,0.081234,0.195101,...,0.077739,0.024537,0.048584,0.013655,0.013230,0.039832,0.334317,0.061355,0.286600,0.021554
min,39.000000,0.726373,0.319748,0.001793,0.286114,0.273420,0.164244,0.488766,0.329020,0.005374,...,0.639517,0.841841,0.532597,0.885433,0.884930,0.729636,0.002086,0.002974,0.018531,0.826599
25%,2572.000000,0.861443,0.461916,0.006733,0.694787,0.754696,0.779383,0.549240,0.430698,0.064762,...,0.753700,0.917521,0.625970,0.947596,0.960867,0.846885,0.135215,0.071464,0.250035,0.899496
50%,5093.000000,0.897031,0.575877,0.010155,0.752331,0.809565,0.870988,0.577799,0.464426,0.135890,...,0.817455,0.934899,0.654580,0.957980,0.969819,0.876994,0.579495,0.125140,0.493621,0.914778
75%,7482.000000,0.932098,0.700989,0.023336,0.810458,0.884021,0.913039,0.619497,0.529877,0.317957,...,0.889239,0.947154,0.688864,0.966194,0.976128,0.897060,0.811444,0.163958,0.747092,0.929312
max,9640.000000,0.983444,0.893370,0.678393,0.977777,0.987769,0.979449,0.737581,0.752813,0.909086,...,0.954701,0.980427,0.798321,0.983716,0.991860,0.969036,0.971783,0.278746,0.995667,0.965446
